# Implementation of the BB84 Protocol for Quantum Key Distribution


Link to the original code and explanation: https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/Cryptography/QuantumKeyDistribution.ipynb

In crypthography, key exchange between involved parties must happen without losing confidentiality, availability and integrity.
Often, it is necessary to use a public channel to exchange keys, which may be exposed to many risks.
A diffused method for this is ciphering the key with public/asymmetric cripthography methods.
This key will be used to cipher messages, generally using symmetric cripthography methods.

The most used algorithm for key distribution is RSA, but now that we have quantum computers and new algorithms, new security measures are needed.
For example, Shor's algorithm threathens the use of RSA, as it can easily calculate the secret prime numbers that must remain confidential for the security of an RSA key.
Also other asymmetric cripthography methods are now threatened.

Therefore, it's crucial to find new alternatives for key distribution, like the BB84 protocol.

An important advantage of BB84 is given by using a quantum system: if an attacker intercepts the exchange, they cannot read the key, as it would mean measuring it. Measuring would make the system collapse into a classical state. This would lead to a relevant loss of information. Plus, all parties involved would notice the anomaly, unlike it often happens in classical computer science, where you can't always be sure your password has not been discovered by an eavesdropper, for example, until they do something to lead the integrity or the availability of related services.
Noticing the key has been intercepted, another key could be easily encryptend and sent again.
If the key has not been intercepted, all parties involved will have a new secret key, useful to encrypt messages, usually using symmetric cripthography methods.

To make BB84 work correctly, some requirements must be met:
- All parties must have access to their own quantum computer.
- They must have a communication channel capable of transmitting qubits.
- They must have a classical communication channel.

Since it is impossible to ensure perfect security, we must assume that any of these channels can be tapped into by an attacker.

If Alice and Bob want to exchange a key, in a nutshell:

- Alice creates a random string of bits, and for each bit, she randomly chooses a basis to encode it in.
- Alice encodes the bits into qubits using her chosen bases, and sends the qubits over a quantum communication channel to Bob's quantum computer.
- Bob also randomly chooses a basis to decode each qubit in. He measures each qubit in the bases he chose.
- Alice uses a classical communication channel to tell Bob which bases she chose. She also tells him the first few bits she sent.
- Bob analyzes these first few bits to determine whether Eve tapped into their quantum communication channel and intercepted Alice's qubits.
- If Eve did not intercept the qubits, they consider all of the qubits that they happened to choose the same bases for, and use those bits as their key. If Eve did intercept the qubits, they repeat the process all over again.

## 0. Preparation
- Imports
- Preparation of simulator and connection to the real quantum computer

In [1]:
#Importing Qiskit
from qiskit import *
#from qiskit.tools.visualization import plot_bloch_multivector #(deprecated)
from qiskit.visualization import plot_bloch_multivector

#For the execution of the code on the simulator and on a real quantum computer
from qiskit_aer import Aer
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

# importing other useful libraries
import sys
import random
import csv #to save results in a csv file
import datetime #to manage timestamps
import os #to check if a file for saving results already exists

#Constants

#Preparing to save elaborations on csv file
elaborations = [] #this will contain all the dictionaries that will be added with .append
dataForCSVFile = {} #this will contain the current elaboration
dataForCSVFile["KEY_LENGTH"] = 'None'
dataForCSVFile["job_id"] = 'simulator' #default
dataForCSVFile["fina_key_len"] = 'None'
dataForCSVFile["attack_noticed"] = 'not to consider' #default for same reason
dataForCSVFile["timestamp"] = 'not saved'

In [2]:
def check_saved_credentials():
    if (input("Do you want to use an account that is already on the computer? Enter 'y' for yes and 'n' for no") == "n"):
        return input("Enter your token once so it can be used for the whole execution: ")
    else:
        print("Credentials will be checked at the moment of execution.")
        return None

In [3]:
# Choose where to execute the program

print("Run the program on the simulator or on the quantum computer?")
value = input("Enter 0 to use the simulator. Enter 1 to use the first available real quantum computer. Enter a value: ")
print("You entered: ", value)

executeOnWhichDevice = "simulator" #possible values: "simulator" (default), "real_quantum_computer"
given_token = None #default value, only needed with real quantum computer

if value == "1":
    executeOnWhichDevice = "real_quantum_computer"

    #Asks the user to give their token
    given_token = check_saved_credentials()

elif value != "0":
    print("Simulator has been chosen by default for the execution.")

print("Execution will be on: " + executeOnWhichDevice)


Run the program on the simulator or on the quantum computer?
You entered:  0
Execution will be on: simulator


In [4]:
#Choose key length

KEY_LENGTH = 500 #default

print("How long will the key be?")
value = input("Enter an integer or 'e' to use length given in the example. Enter a value: ")
print("You entered: ", value)

if value == "e":
    KEY_LENGTH = 500
else:
    try:
        KEY_LENGTH = int(value)
    except ValueError:
        print("Value not valid. Key length will be chosen by default.")

dataForCSVFile["KEY_LENGTH"] = KEY_LENGTH

print("Length of the key: " + str(KEY_LENGTH))

How long will the key be?
You entered:  e
Length of the key: 500


In [5]:
#Definition of useful functions

def executeOnQuantumMachine_simulator(qc,backend=''):

    # Esecuzione del circuito quantistico
    simulator = Aer.get_backend(backend)
    result = simulator.run(qc).result()
    
    return result

def executeOnRealQuantumComputer(qc):

    try:
        
        service = None

        if given_token is not None:
            service = QiskitRuntimeService(channel="ibm_quantum", token=given_token)
        else:
            service = QiskitRuntimeService(channel="ibm_quantum")

        backend = service.least_busy(operational=True, simulator=False)

        # Trasponi il circuito per il backend
        qc_transpiled = transpile(qc, backend)

        sampler = Sampler(backend)

        print("The next job will begin...")

        
        job = sampler.run([qc_transpiled])
        print(f"job id: {job.job_id()}")
        dataForCSVFile["job_id"] = job.job_id()
        result = job.result()

        print("Current job ended. Returning the result...")

        return result
    
    except Exception as e:
        print("An error occurred:")
        print(e)
        return None

def executeCode(executeOnWhichDevice, qc, backend):
    #possible values: "simulator" (default), "real_quantum_computer"
    if qc == None:
        print("Empty circuit, returning None")
        return None
    elif executeOnWhichDevice == "simulator":
        return executeOnQuantumMachine_simulator(qc,backend)
    elif executeOnWhichDevice == "real_quantum_computer":
        return executeOnRealQuantumComputer(qc)
    else:
        print("Simulator has been chosen by default for the execution.")
        return executeOnQuantumMachine_simulator(qc,'qasm_simulator')


In [6]:
FILE_NAME = str(input("Insert the name of the file (including the path) where to save your results (pressing enter will lead to use the default location, the current folder, and a default name for your file: results_of_executions.csv):"))
if not FILE_NAME:
    FILE_NAME = os.getcwd() + "\\results_of_executions.csv"

# Writing (appending to already existing recorded elaborations) to csv file given the name of the file and a list of dictionaries
def writeOnCSVFile(filename, elaborations):
    print("Writing on " + filename + "...")
    try:
        mode = 'a' if os.path.exists(filename) else 'w'
        with open(filename, mode) as csvfile:
            # Create a csv writer object
            writer = csv.DictWriter(csvfile, fieldnames = elaborations[0].keys())
            
            if mode == 'w':
                # Write the header only if a new file is created
                writer.writeheader()
            
            # Write the data
            for data in elaborations:
                writer.writerow(data)
        print("Data saved correctly into " + filename + " ")
    except Exception as e:
        print("Error: " + str(e))

# A function to save timestamps
def saveTimestamp():
    # Get the current date and time
    now = datetime.datetime.now()

    # Format the date and time as a string
    timestamp = now.strftime("%Y-%m-%d %H:%M:%S")

    return timestamp


## 1. Choosing a base and encrypting the key

The first step is for Alice to choose a string of bits and bases to encode them in.
We can represent a qubit as a vector on a "Bloch Sphere". Each axis can be considered a possible bases.


In [7]:
qc = QuantumCircuit(2)
qc.x(1)
try:
    result = executeCode("simulator",qc,'statevector_simulator') #statevector only works on simulator
    statevector = result.get_statevector()
    plot_bloch_multivector(statevector)
except Exception as e:
    print("Error in generating bloch sphere 1")
    print(e)

Another base we can use is the X-basis, which lies on the X-axis.

In [8]:
qc = QuantumCircuit(2)
qc.h(0)
qc.x(1)
qc.h(1)
try:
    result = executeCode("simulator",qc,'statevector_simulator')
    statevector = result.get_statevector()
    plot_bloch_multivector(statevector)
except Exception as e:
    print("Error in generating bloch sphere 1")
    print(e)

Now, we have two different ways to encode a 0, and two different ways to encode a 1.
Alice randomly creates a string of, for example, 500 bits. (EDIT: But you can also choose another value at the beginning of the execution)
She can do this by flipping a coin and writing down 0 everytime she lands on heads and 1 everytime she lands on tails.


In [9]:
# Preparation for encoding
#KEY_LENGTH = 500 #the value of the key in the example

#random.seed(0) # Seed the random number generator. This will be used as our "coin flipper".

# Generating a random string of bits
alice_bits = ""
for i in range(KEY_LENGTH):
    randBit = random.randint(0, 1) # Flip Coin
    alice_bits += str(randBit) # Add randomly chosen bit to the bit string.

print("The bits Alice is going to send are: " + alice_bits[:10] + "...")

The bits Alice is going to send are: 1000011110...


Then, Alice randomly chooses a basis of each bit.
She can do this by flipping a coin and writing down Z everytime she lands on heads and X everytime she lands on tails.


In [10]:
def generate_random_bases(num_of_bases):
    """This function selects a random basis for each bit"""
    bases_string = ""
    for i in range(num_of_bases):
        randBasis = random.randint(0, 1) # Flip Coin

        if randBasis == 0:
            bases_string += "Z" 
        else:
            bases_string += "X"
            
    return bases_string

alice_bases = generate_random_bases(KEY_LENGTH) # Alice randomly chooses a basis for each bit.

print("The bases Alice is going to encode them in are: " + alice_bases[:10] + "...")

The bases Alice is going to encode them in are: XXZZXXXXZZ...


Now, Alice encodes each bit into its corresponding basis on her quantum computer, creating a string of 500 qubits. She sends these qubits over an optical cable to Bob.

By default, quantum computers measure in the Z-basis and all qubits are initialized to |0>. To turn this into an |1>, we need to apply an X gate.

To encode in the X-basis, we start with the corresponding |0> or |1> and then apply a Hadamard gate to convert |0> in |+> and |1> into |->.

- | Bit: 0 | Basis: Z | Qubit state: |0> | Gate required: none     |
- | Bit: 1 | Basis: Z | Qubit state: |1> | Gate required: X        |
- | Bit: 0 | Basis: X | Qubit state: |+> | Gate required: H        |
- | Bit: 1 | Basis: X | Qubit state: |-> | Gate required: X then H |

We'll store the quantum circuit used to encode each qubit.

In [11]:
def encode(bits, bases):
    #This function encodes each bit into the given basis.
    
    encoded_qubits = []
    
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1) # Create a quantum circuit for each qubit
        
        # Possible Cases
        if bit=="0" and basis == "Z":
            encoded_qubits.append(qc) # Do not apply any gates

        elif bit=="1" and basis == "Z":
            qc.x(0) # Apply X Gate
            encoded_qubits.append(qc)

        elif bit=="0" and basis == "X":
            qc.h(0) # Apply H Gate
            encoded_qubits.append(qc)

        elif bit=="1" and basis == "X":
            qc.x(0) # Apply X Gate
            qc.h(0) # Apply H Gate
            encoded_qubits.append(qc)
            
    return (encoded_qubits)

# Encode Alice's bits
encoded_qubits = encode(alice_bits, alice_bases)

# Print circuits for first 5 qubits.
for i in range(5):
    print(encoded_qubits[i])
print("etc.")

     ┌───┐┌───┐
  q: ┤ X ├┤ H ├
     └───┘└───┘
c: 1/══════════
               
     ┌───┐
  q: ┤ H ├
     └───┘
c: 1/═════
          
     
  q: 
     
c: 1/
     
     
  q: 
     
c: 1/
     
     ┌───┐
  q: ┤ H ├
     └───┘
c: 1/═════
          
etc.


## 2. Sending the qubits

Normally, Alice would send these encoded qubits over a quantum channel, such as an optical fibre cable. However, since we don't have quantum channels in the real world yet, we'll store the circuits for each qubit in an array called QUANTUM_CHANNEL for the sake of demonstration. This would be like if Alice stored the qubits in a building and waited for Bob to pick them up. Of course, just like a fibre optic cable, this building is also a location where Eve might intercept the message.

In [12]:
QUANTUM_CHANNEL = encoded_qubits

## 3. Measurement

The next step is for Bob to receive the encoded qubits and measure them. First, he must choose his own set of random bases, just like Alice.

In [13]:
bob_bases = generate_random_bases(KEY_LENGTH) # Bob randomly chooses a basis for each bit.

print("The bases Bob is going to decode them in are: " + bob_bases[:10] + "...")

The bases Bob is going to decode them in are: XZXZXZXXZZ...


Then, Bob must measure each Qubit in the corresponding bases he chose. In Qiskit, this can be accomplished by adding a measurement gate to the circuit for each encoded qubit, and then executing it. However, IBM's quantum computers measure in the Z-basis by default, so to measure in the X-basis we must apply a Hadamard gate before our measurement gate.

In [14]:
def measure(qubits, bases):
        """This function measures each qubit in the corresponding basis chosen for it."""

        bits = "" # The results of measurements

        for qubit, basis in zip(qubits, bases):

            # Add measurement depending on basis
            if basis == "Z":
                qubit.measure(0, 0)
            elif basis == "X":
                qubit.h(0)
                qubit.measure(0, 0)
            
            measured_bit = None

            result = executeCode(executeOnWhichDevice,qubit,'qasm_simulator') #'qasm_simulator' is only needed when executeOnWhichDevice="simulator"

            if result is None:
                print("An error occurred during the execution. Stopping the execution of the program...")
                sys.exit(1) #in case of an error, the program will be terminated and you'll need to execute it again from the start.

            if executeOnWhichDevice=="real_quantum_computer":

                print(len(qubits))
                c = ClassicalRegister(len(qubits), 'c')

                counts = result[0].data.c.get_counts()
                measured_bit = max(counts, key=counts.get) # Max doesn't matter for simulator since there is only one shot.
            
            else:
                counts = result.get_counts()
                measured_bit = max(counts, key=counts.get) # Max doesn't matter for simulator since there is only one shot.

            bits += measured_bit
            
        return bits


qubits_received = QUANTUM_CHANNEL # Receive qubits from quantum channel
bob_bits = measure(qubits_received, bob_bases)

print("The first few bits Bob received are: " + bob_bits[:10] + "...")



The first few bits Bob received are: 1000011110...


## 4. Comparison

Now, Alice needs to tell Bob which bases she chose to encode her qubits in. She can tell him over any classical channel. The beauty of this protocol is that it doesn't matter if Eve finds out which bases Alice used.

In [15]:
CLASSICAL_CHANNEL = alice_bases # Alice tells Bob which bases she used

For each of the qubits where Alice and Bob chose different bases, there is a 50% chance Bob's measurement returned the wrong qubit. For example, If Alice sent Bob a qubit in the |+> state (i.e., a bit value of 0 encoded in the X-basis), and Bob measures in the Z-basis, there is a 50% chance he will get a 
|0> and a 50% chance he will get a |1>. Thus, every instance where their bases don't match is useless to them, so Bob needs to find the bases they share in common.

In [16]:
# Store the indices of the bases they share in common
common_bases = [i for i in range(KEY_LENGTH) if CLASSICAL_CHANNEL[i]==bob_bases[i]]

print("The indices of the first 10 bases they share in common are: " + str(common_bases[:10]))

The indices of the first 10 bases they share in common are: [0, 3, 4, 6, 7, 8, 9, 10, 11, 13]


Now that Bob knows the bases they share in common, he can discard all the rest of the bits, and only keep the ones that were measured in the same bases.

In [17]:
bob_bits = [bob_bits[index] for index in common_bases]


He also tells Alice which bases they had in common, so that she can discard the rest of the bits as well, keeping only the bits that were measured in the same bases that she encoded them in.

In [18]:
CLASSICAL_CHANNEL = common_bases # Bob tells Alice which bases they shared in common
alice_bits = [alice_bits[index] for index in common_bases] # Alice keeps only the bits they shared in common

Since Alice and Bob are only keeping the bits measured in the bases they shared in common, they should have the same bits. To make sure this is the case, Alice will announce the first few bits that she has, and Bob should have the same ones. Of course, if Eve were trying to eavesdrop, she would also hear these first few bits, so Alice and Bob would have to discard them as well (after comparing to make sure they're the same as what they expect).

In [19]:
CLASSICAL_CHANNEL = alice_bits[:100] # Alice tells Bob the first 100 bits she has left.

# Bob checks if they match the first 100 bits that he has
if CLASSICAL_CHANNEL == bob_bits[:100]:
    print("Yep, Alice and Bob seem to have the same bits!")
else:
    print("Uh oh, at least one of the bits is different.")

Yep, Alice and Bob seem to have the same bits!


Since the first 100 bits are the same, Alice and Bob can be fairly certain that the remaining bits also match. Now, they need to discard the first 100 bits, since Eve may have been listening in on the classical channel and keeping track of what they are.

In [20]:
alice_bits = alice_bits[100:] # Alice discards the first 100 bits
bob_bits = bob_bits[100:] # Alice discards the first 100 bits

These remaining bits are the key that Alice and Bob need to set up an encrypted communication channel! Now they can communicate without worrying about their messages being read by someone else.

In [21]:
key = "" 
for bit in alice_bits: # Or bob_bits, since both should be the same
    key += bit
    
dataForCSVFile["fina_key_len"] = len(key)

print("Shhhhh, the key is:")
print(str(key))
print("Don't tell anyone!")

print("\nThe key is " + str(len(key)) + " bits long.")

dataForCSVFile["timestamp"] = saveTimestamp()
writeOnCSVFile(FILE_NAME,[dataForCSVFile])

Shhhhh, the key is:
1000000101000111010001100100100000110100011000001111011100100111000000001100000100100011100010101111010111100011000111000010011001110010100001101100100010001000110011
Don't tell anyone!

The key is 166 bits long.
Writing on C:\Users\ruggy\qiskit_projects\per_esame\csv files (older executions)\results_of_executions_8th_file.csv...
Data saved correctly into C:\Users\ruggy\qiskit_projects\per_esame\csv files (older executions)\results_of_executions_8th_file.csv 


The length of the key varies depending on how many bases they share in common.

# Interception

So far, we've only looked at the case where no one spying on Alice and Bob. But what if someone like Eve tries to intercept their key? Let's see what happens in this case.

First, Alice performs the encoding step as usual: she randomly generates a string of bits and bases to encode them in. She then sends the encoded qubits along a quantum channel (fibre-optic cable) to Bob.

In [22]:
# Generating a random string of bits
alice_bits = ""
for i in range(KEY_LENGTH):
    randBit = random.randint(0, 1) # Flip Coin
    alice_bits += str(randBit) # Add randomly chosen bit to the bit string.

# Alice randomly chooses a basis for each bit.
alice_bases = generate_random_bases(KEY_LENGTH)

# Encode Alice's bits
encoded_qubits = encode(alice_bits, alice_bases)

QUANTUM_CHANNEL = encoded_qubits

## Interception by Eve
This time, the qubits are intercepted by Eve. Let's show what she would do. First, she would randomly choose a set of bases to measure the qubits in (since she has no idea which bases Alice used). Then, she'll perform the measurements. This is similar to what Bob would normally do.

In [23]:
qubits_intercepted = QUANTUM_CHANNEL # Intercept qubits
eve_bases = generate_random_bases(KEY_LENGTH) # Generate a random set of bases
eve_bits = measure(qubits_intercepted, eve_bases) # Measure the qubits

Because of the No-Cloning Theorem of Quantum Mechanics, Eve cannot just copy the qubits over from the quantum channel. Thus, Bob will never receive the qubits, making it obvious to him and Alice that their message was intercepted. To prevent them from realizing what has happened, Eve must create her own decoy qubits to send to Bob. Since she has no idea which bases Alice used, she has to generate the bases randomly. This is similar to what Alice originally did when she encoded the qubits.

Eve might generate an entirely new set of bases, or she might just use the same ones she used to measure the qubits. For simplicity, let's assume the bases she used to intercept the qubits are the same bases she uses to encode her decoy qubits.

In [24]:
# Eve encodes her decoy qubits and sends them along the quantum channel
QUANTUM_CHANNEL = encode(eve_bits, eve_bases)

Bob then performs his usual steps: selecting a measurement bases and measuring the qubits. He doesn't know that Eve has intercepted them yet.

In [25]:
bob_bases = generate_random_bases(KEY_LENGTH) # Bob randomly chooses a basis for each bit.
qubits_received = QUANTUM_CHANNEL # Receive qubits from quantum channel
bob_bits = measure(qubits_received, bob_bases)

## Comparison
Again, Alice needs to tells Bob which bases she chose to encode her qubits in. She can tells him over any classical channel (perhaps even publicly on Twitter). Since this classical channel is public, Eve also knows which bases she used.

In [26]:
CLASSICAL_CHANNEL = alice_bases # Alice tells Bob which bases she used

As usual, Bob checks which bases they share in common. He discards the bits that were not measured in the same bases and keeps the ones that were measured in the same bases Alice encoded them in.

In [27]:
# Store the indices of the bases they share in common
common_bases = [i for i in range(KEY_LENGTH) if CLASSICAL_CHANNEL[i]==bob_bases[i]]
bob_bits = [bob_bits[index] for index in common_bases]

He then tells Alice the indices of the bases they shared in common so that she can do the same.

In [28]:
CLASSICAL_CHANNEL = common_bases # Bob tells Alice which bases they shared in common
alice_bits = [alice_bits[index] for index in common_bases] # Alice keeps only the bits they shared in common

## Waiiiiiiit a Second...
Since Alice and Bob are only keeping the bits measured in the bases they shared in common, they should have the same bits. To make sure this is the case, Alice will announce the first few bits that she has, and Bob should have the same ones.

In [29]:
CLASSICAL_CHANNEL = alice_bits[:100] # Alice tells Bob the first 100 bits she has left.

# Bob checks if they match the first 100 bits that he has
if CLASSICAL_CHANNEL == bob_bits[:100]:
    print("Yep, Alice and Bob seem to have the same bits!")
    dataForCSVFile["attack_noticed"] = 'false'
else:
    print("Uh oh, at least one of the bits is different.")
    dataForCSVFile["attack_noticed"] = 'true'

dataForCSVFile["timestamp"] = saveTimestamp()
writeOnCSVFile(FILE_NAME,[dataForCSVFile])

Uh oh, at least one of the bits is different.
Writing on C:\Users\ruggy\qiskit_projects\per_esame\csv files (older executions)\results_of_executions_8th_file.csv...
Data saved correctly into C:\Users\ruggy\qiskit_projects\per_esame\csv files (older executions)\results_of_executions_8th_file.csv 


After comparing the first 100 bits, they see that their bits don't match! Assuming that their is no noise in the quantum channel and quantum computers (which could cause errors), Alice and Bob can be certain that their message was intercepted! Thus, they can throw away all their bits and repeat the same protocol all over again. This time, they might use a different quantum channel in an effort to throw Eve off!

# Sources
- Qiskit Textbook: https://qiskit.org/textbook/ch-algorithms/quantum-key-distribution.html
- Qubit by Qubit's Introduction to Quantum Computing Course